# Experiment A - Random Time Schedules

In [1]:
from ipywidgets import AppLayout, GridspecLayout
from ipywidgets import interact
import pandas as pd
import numpy as np



from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, GridspecLayout
from ipywidgets import FloatSlider
import json
from bqplot import OrdinalScale, LinearScale, Lines, Axis, Figure, Scatter, Label
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot
import pandas as pd

from IPython.display import display

In [3]:
store = pd.HDFStore('./min_run.h5')
min_stations = store['stations']
min_trains = store['train']
min_tracks = store['track']
store.close()

store = pd.HDFStore('./max_run.h5')
max_stations = store['stations']
max_trains = store['train']
max_tracks = store['track']
store.close()


In [5]:
import matplotlib.pyplot as plt 
plt.rcParams['figure.dpi'] = 150

class TrainPloter():
    
    def __init__(self, id, titel, trains, stations, tracks):
        self.id = id
        self.trains = trains
        self.stations = stations
        self.tracks = tracks
        self.titel = titel
    
    def plot_station(self, ds):
        plt.scatter(ds['Location'][0], ds['Location'][1], marker="s", zorder=15)
        plt.text(ds['Location'][0], ds['Location'][1]+340, ds['Id'], fontsize=9, zorder=21)
        plt.text(ds['Location'][0], ds['Location'][1] + 120, '{} | {}'.format(ds['PassengersWaiting'], ds['PassengersHandled']), fontsize=9, zorder=21)
        
    def plot_train(self, ds):
        plt.scatter(ds['Location'][0], ds['Location'][1], marker="x", zorder=20)
        plt.text(ds['Location'][0] + 20, ds['Location'][1]- 150, ds['Id'], fontsize=9, zorder=21)
        plt.text(ds['Location'][0] + 20, ds['Location'][1] - 350, '{}/{}'.format(ds['PassengerLevel'], ds['Capacity']), fontsize=9, zorder=21)
         
    def plot_track(self, ds):
        plt.plot(ds['PositionA'],ds['PositionB'],'-', zorder=1, color='grey', linewidth=ds['Capacity'])
    
    
    def plot_simulation(self, timeslot):
        plt.figure(self.id, figsize=(7,4))
        plt.clf()
         
        for ds in self.trains[timeslot].iloc:
            self.plot_train(ds)
            
        for ds in self.stations[timeslot].iloc:
            self.plot_station(ds)
        
        for ds in self.tracks[timeslot].iloc:
            self.plot_track(ds)
        plt.title(self.titel)
        plt.plot()

In [6]:
grid_anim = GridspecLayout(10,3)

play = widgets.Play(
    value=0,
    min=0,
    max=1420,
    step=1,
    interval=100,
    description="Press play",
    disabled=False,
    layout=Layout(width='auto', height='auto')
)
slider = widgets.IntSlider(min=0, max=1420, step=1, layout=Layout(width='auto', height='auto'), continuous_update=False)
widgets.jslink((play, 'value'), (slider, 'value'))


min_ploter = TrainPloter(1, 'Network with worst overall Reward\n', min_trains, min_stations, min_tracks)
max_ploter = TrainPloter(2, 'Network with best overall Reward\n', max_trains, max_stations, max_tracks)

min_output_network = widgets.interactive_output(min_ploter.plot_simulation, {'timeslot': slider})
max_output_network = widgets.interactive_output(max_ploter.plot_simulation, {'timeslot': slider})



grid_anim[0:9,0] = min_output_network
grid_anim[0:9,1] = max_output_network
grid_anim[9,0:1] = widgets.VBox([slider, play], layout=Layout(width='auto', height='auto'))

grid_anim

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

<br>

## <center>Overview Reward Function </center>

In [12]:
def create_quantile_plot(data, subdata: str, x_lable: str, title: str=''):
    
    head = list(data[0][subdata].keys())
    df = pd.DataFrame([tuple(d[subdata].values()) for d in data], columns=head)
    df= (df/2.0)*100
    x = head[:-1]
    
    quantiles_10 = [n for n in df[:].quantile(0.1)][:-1]
    quantiles_25 = [n for n in df[:].quantile(0.25)][:-1]
    quantiles_50 = [n for n in df[:].quantile(0.5)][:-1]
    quantiles_75 = [n for n in df[:].quantile(0.75)][:-1]
    quantiles_90 = [n for n in df[:].quantile(0.9)][:-1]
    
    
    x_ord = OrdinalScale()
    y_sc = LinearScale(min=0, max=100)
    
    line_q10 = Lines(x=x, y=quantiles_10, scales={'x': x_ord, 'y': y_sc}, stroke_width=2, colors=['#758eff'], display_legend=True, labels=['10% Quantile'])
    line_q25 = Lines(x=x, y=quantiles_25, scales={'x': x_ord, 'y': y_sc}, stroke_width=2, colors=['#758eff'], line_style='dotted',display_legend=True, labels=['25% Quantile'])
    line_q50 = Lines(x=x, y=quantiles_50, scales={'x': x_ord, 'y': y_sc}, stroke_width=2, colors=['#3f59cf'], display_legend=True, labels=['50% Quantile'])
    line_q75 = Lines(x=x, y=quantiles_75, scales={'x': x_ord, 'y': y_sc}, stroke_width=2, colors=['#0422a7'], line_style='dotted', display_legend=True, labels=['75% Quantile'])
    line_q90 = Lines(x=x, y=quantiles_90, scales={'x': x_ord, 'y': y_sc}, stroke_width=2, colors=['#0422a7'], display_legend=True, labels=['90% Quantile'])
    
    ax_x = Axis(scale=x_ord, grid_lines='solid', label=x_lable)
    ax_y = Axis(scale=y_sc, grid_lines='dashed', orientation='vertical', label='% of maximum Reward')
    
    return Figure(marks=[line_q90, line_q75, line_q50, line_q25, line_q10], axes=[ax_x, ax_y], title=title, legend_location='bottom-left')

In [13]:

grid = GridspecLayout(1,3)
#plt.tight_layout()


with open('../experiments/data3.json') as json_file:
    data = json.load(json_file)


fig1 = create_quantile_plot(data, 'trains_increase', 'Number of trains per h per line')
fig2 = create_quantile_plot(data, 'passenger_increase', 'Multiple of base passenger demand')
fig3 = create_quantile_plot(data, 'trains_delay', 'Duration of single train breakdown in system')

grid[0, 0] = create_quantile_plot(data, 'trains_increase', 'Number of trains per h per line', 'Change of Reward with increasing amount of trains')
grid[0, 1] = create_quantile_plot(data, 'passenger_increase', 'Multiple of base passenger demand', 'Change of Reward with increasing demand ')
grid[0, 2] = create_quantile_plot(data, 'trains_delay', 'Duration of single train breakdown', 'Change of Reward with increasingly long breakdown of a single train')
grid


GridspecLayout(children=(Figure(axes=[Axis(label='Number of trains per h per line', scale=OrdinalScale()), Axi…